# MLFlow Model Server


Clone MLflow repo so we get access to the examples

In [1]:
!git clone https://github.com/mlflow/mlflow

Cloning into 'mlflow'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 11829 (delta 0), reused 0 (delta 0), pack-reused 11828
Receiving objects: 100% (11829/11829), 9.69 MiB | 1.17 MiB/s, done.
Resolving deltas: 100% (7778/7778), done.


Run the sklearn elasticnet wine example so we generate some mlflow models, which we can then load

In [2]:
!python mlflow/examples/sklearn_elasticnet_wine/train.py

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.8222428497595403
  MAE: 0.6278761410160693
  R2: 0.12678721972772622


## Test locally

In [7]:
import sys

sys.path.append("..")
import os

from mlflowserver.MLFlowServer import MLFlowServer

In [11]:
m = MLFlowServer("mlruns/0/" + next(os.walk("mlruns/0"))[1][0] + "/artifacts/model")
m.load()

In [12]:
m.predict([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]], feature_names=[])

array([6.24456105])

## REST test

In [27]:
!cd .. && make build_rest

s2i build -E environment_rest ./mlflowserver seldonio/seldon-core-s2i-python37:0.11-SNAPSHOT seldonio/mlflowserver_rest:0.1
---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl


Stored in directory: /root/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Stored in directory: /root/.cache/pip/wheels/8b/65/b2/9837b4422d13e739c3324c428f1b3aa9e3c3df666bb420e4b3
Stored in directory: /root/.cache/pip/wheels/5d/1a/1e/0350bb3df3e74215cd91325344cc86c2c691f5306eb4d22c77
Stored in directory: /root/.cache/pip/wheels/1e/41/34/23ebf5d1089a9aed847951e0ee375426eb4ad0a7079d88d41e
Stored in directory: /root/.cache/pip/wheels/f2/ec/e0/d7deb0c981557e373edf7370574b7001690892afe5fea30c3c
Stored in directory: /root/.cache/pip/wheels/98/32/7b/a291926643fc1d1e02593e0d9e247c5a866a366b8343b7aa27
Stored in directory: /root/.cache/pip/wheels/2b/67/89/414471314a2d15de625d184d8be6d38a03ae1e983dbda91e84
Successfully built sklearn alembic simplejson querystring-parser sqlalchemy Mako tabulate
You should consider upgrading via the 'pip install --upgrade pip' command.
Build completed successfully


In [17]:
print("mlruns/0/" + next(os.walk("mlruns/0"))[1][0] + "/artifacts/model")

mlruns/0/640ee112155e46e682b35b2768ae7f4d/artifacts/model


In [35]:
!echo `python -c 'import os; print("mlruns/0/"+next(os.walk("mlruns/0"))[1][0]+"/artifacts/model")'`

mlruns/0/640ee112155e46e682b35b2768ae7f4d/artifacts/model


In [45]:
!docker rm -f "mlflowserver"

mlflowserver


In [42]:
!docker run --rm -d --name "mlflowserver"  -p 5000:5000 -e PREDICTIVE_UNIT_PARAMETERS='[{"type":"STRING","name":"model_uri","value":"file:///model"}]' -v `pwd`/`python -c 'import os; print("mlruns/0/"+next(os.walk("mlruns/0"))[1][0]+"/artifacts/model")'`:/model seldonio/mlflowserver_rest:0.1

6b66cf04d948d78f8b856f8de792201a0d08a11bfdf5e67d1749f5255eb383e0


Send some random features that conform to the contract

In [46]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p

----------------------------------------
SENDING NEW REQUEST:

[[6.666 3.774 0.138 0.858 2.137 1.385 2.822 1.194 0.228 1.692 1.228]]
RECEIVED RESPONSE:
meta {
}
data {
  ndarray {
    values {
      number_value: 5.285221887024144
    }
  }
}




In [47]:
!docker rm mlflowserver --force

mlflowserver


## grpc test

In [49]:
!cd .. && make build_grpc

s2i build -E environment_grpc ./mlflowserver seldonio/seldon-core-s2i-python37:0.11-SNAPSHOT seldonio/mlflowserver_grpc:0.1
---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl


Stored in directory: /root/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Stored in directory: /root/.cache/pip/wheels/5d/1a/1e/0350bb3df3e74215cd91325344cc86c2c691f5306eb4d22c77
Stored in directory: /root/.cache/pip/wheels/8b/65/b2/9837b4422d13e739c3324c428f1b3aa9e3c3df666bb420e4b3
Stored in directory: /root/.cache/pip/wheels/f2/ec/e0/d7deb0c981557e373edf7370574b7001690892afe5fea30c3c
Stored in directory: /root/.cache/pip/wheels/1e/41/34/23ebf5d1089a9aed847951e0ee375426eb4ad0a7079d88d41e
Stored in directory: /root/.cache/pip/wheels/2b/67/89/414471314a2d15de625d184d8be6d38a03ae1e983dbda91e84
Stored in directory: /root/.cache/pip/wheels/98/32/7b/a291926643fc1d1e02593e0d9e247c5a866a366b8343b7aa27
Successfully built sklearn simplejson alembic sqlalchemy querystring-parser tabulate Mako
You should consider upgrading via the 'pip install --upgrade pip' command.
Build completed successfully


In [58]:
!docker rm -f "mlflowserver"

mlflowserver


In [51]:
!docker run --rm -d --name "mlflowserver"  -p 5000:5000 -e PREDICTIVE_UNIT_PARAMETERS='[{"type":"STRING","name":"model_uri","value":"file:///model"}]' -v `pwd`/`python -c 'import os; print("mlruns/0/"+next(os.walk("mlruns/0"))[1][0]+"/artifacts/model")'`:/model seldonio/mlflowserver_grpc:0.1

cbfcb010d313d3ebb74a34933fa05b2ab7cbbfe43934a47e6c9fd4f7c08c64e6


Test using NDArray payload

In [64]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p --grpc

----------------------------------------
SENDING NEW REQUEST:

[[7.51  8.353 2.899 1.155 0.458 1.462 2.619 0.378 1.856 1.969 1.361]]
RECEIVED RESPONSE:
None



Test using Tensor payload

In [54]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p --grpc --tensor

----------------------------------------
SENDING NEW REQUEST:

[[7.522 4.718 2.436 0.608 1.946 2.264 2.842 2.468 1.831 0.618 0.01 ]]
RECEIVED RESPONSE:
None



In [11]:
!docker rm sklearnserver --force

sklearnserver
